<a href="https://colab.research.google.com/github/10dimensions/large-notebook-repository/blob/master/VideoPose_MediaPipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip uninstall tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/lib/python3.12/dist-packages/tensorflow-2.19.0.dist-info/*
    /usr/local/lib/python3.12/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.19.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tenso

In [2]:
# Cell 1: Install dependencies
!pip install yt-dlp mediapipe opencv-python matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing inst

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import os
import yt_dlp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [2]:

# Replace with your YouTube URL
youtube_url = "https://www.youtube.com/watch?v=VHl8nUfUhyw"  # Update this!

ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    'outtmpl': 'downloaded_video.%(ext)s',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])

video_filename = 'downloaded_video.mp4'  # Adjust if extension differs



[youtube] Extracting URL: https://www.youtube.com/watch?v=VHl8nUfUhyw
[youtube] VHl8nUfUhyw: Downloading webpage
[youtube] VHl8nUfUhyw: Downloading tv client config
[youtube] VHl8nUfUhyw: Downloading tv player API JSON
[youtube] VHl8nUfUhyw: Downloading web safari player API JSON
[youtube] VHl8nUfUhyw: Downloading player 0004de42-main
[youtube] VHl8nUfUhyw: Downloading m3u8 information
[info] VHl8nUfUhyw: Downloading 1 format(s): 399+140
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: downloaded_video.f399.mp4
[download] 100% of    2.75MiB in 00:00:00 at 6.70MiB/s   
[download] Destination: downloaded_video.f140.m4a
[download] 100% of  302.41KiB in 00:00:00 at 1.08MiB/s   
[Merger] Merging formats into "downloaded_video.mp4"
Deleting original file downloaded_video.f140.m4a (pass -k to keep)
Deleting original file downloaded_video.f399.mp4 (pass -k to keep)


In [8]:
print(os.getcwd())

/content


In [17]:
# Cell 4: Extract frames and detect 2D poses

cap = cv2.VideoCapture("downloaded_video.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#print(frame_height)
#print(frame_width)

print(total_frames)

# Store 2D landmarks (33 keypoints per frame)
landmarks_2d = []
with mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=False, min_detection_confidence=0.5) as pose:
    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        print(ret, frame)
        #if not ret:
        #    break

        # Flip frame horizontally for selfie view
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = pose.process(frame)

        if results.pose_landmarks:
            lm = np.array([[lm.x * frame_width, lm.y * frame_height, lm.z] for lm in results.pose_landmarks.landmark])
            landmarks_2d.append(lm)
        else:
            # Pad with zeros if no detection
            landmarks_2d.append(np.zeros((33, 3)))

        frame_idx += 1
        if frame_idx % 30 == 0:  # Progress every 30 frames
            print(f"Processed {frame_idx}/{total_frames} frames")

    cap.release()

print(f"Detected poses in {len(landmarks_2d)} frames")



571
False None


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:

def smooth_poses(poses_2d, window_size=5):
    smoothed = []
    for i in range(len(poses_2d)):
        start = max(0, i - window_size // 2)
        end = min(len(poses_2d), i + window_size // 2 + 1)
        window = poses_2d[start:end]
        smoothed_pose = np.mean(window, axis=0)
        # Scale z for better 3D visualization (pseudo-metric)
        smoothed_pose[:, 2] *= 2000  # Arbitrary scaling
        smoothed.append(smoothed_pose)
    return np.array(smoothed)

landmarks_3d = smooth_poses(landmarks_2d)


In [ ]:

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# MediaPipe pose connections
connections = [
    (0,1),(0,2),(1,3),(2,4),(0,5),(0,6),(5,7),(7,9),(6,8),(8,10),
    (5,11),(6,12),(11,13),(13,15),(12,14),(14,16),(11,23),(12,24),(23,24)
]

def update(frame):
    ax.clear()
    ax.set_xlim(-100, frame_width + 100)
    ax.set_ylim(-100, frame_height + 100)
    ax.set_zlim(-500, 500)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (Depth)')
    ax.set_title(f'3D Pose Estimation - Frame {frame}')

    pose = landmarks_3d[frame]
    # Plot joints
    ax.scatter(pose[:, 0], pose[:, 1], -pose[:, 2], c='b', s=20)  # Invert Z for view

    # Plot skeleton
    for conn in connections:
        if not np.all(pose[conn] == 0):
            ax.plot(pose[conn, 0], pose[conn, 1], -pose[conn, 2], 'r-')

ani = FuncAnimation(fig, update, frames=len(landmarks_3d), interval=1000/fps, repeat=True)
HTML(ani.to_jshtml())

In [ ]:

np.savetxt('3d_poses.csv', landmarks_3d.reshape(len(landmarks_3d), -1), delimiter=',')
from google.colab import files
files.download('3d_poses.csv')